# Research Questions
Actionable guidance의 effectiveness evaluation
1. flippability
2. correctness
3. applicability

## RQ1. 각 approach의 proposed changes는 defect가 예측된 코드에 대한 decision을 얼마나 flip하는가?

### Ex1. Minimal Changes' flip rate
Total: the number of true positives of defect instances in the test set

| Project    | Total | LIME-HPO               | TimeLIME               | SQAPlanner - confidence | SQAPlanner - coverage   | SQAPlanner - lift       |
|------------|-------|------------------------|------------------------|-------------------------|-------------------------|-------------------------|
| activemq@0 | 46    | 13 (28.26%)            | 2 (4.35%)              | 23 (50%)                | 23 (50%)                | 23 (50%)                |
| activemq@1 | 37    | 23 (62.16%)            | 26 (70.27%)            | 10 (27.03%)             | 9 (24.32%)              | 10 (27.03%)             |
| activemq@2 | 67    | 20 (29.85%)            | 24 (35.82%)            | 28 (41.79%)             | 10 (14.93%)             | 28 (41.79%)             |
| activemq@3 | 79    | 20 (25.32%)            | 11 (13.92%)            | 21 (26.58%)             | 16 (20.25%)             | 21 (26.58%)             |
| camel@0   | 38    | 26 (68.42%)            | 6 (15.79%)             | 13 (34.21%)             | 14 (36.84%)             | 13 (34.21%)             |
| camel@1   | 31    | 16 (51.61%)            | 18 (58.06%)            | 4 (12.9%)               | 4 (12.9%)               | 4 (12.9%)               |
| camel@2   | 21    | 13 (61.9%)             | 16 (76.19%)            | 0 (0%)                  | 0 (0%)                  | 0 (0%)                  |
| derby@0   | 395   | 67 (16.96%)            | 59 (14.94%)            | 93 (23.54%)             | 50 (12.66%)             | 92 (23.29%)             |
| derby@1   | 164   | 42 (25.61%)            | 35 (21.34%)            | 80 (48.78%)             | 31 (18.9%)              | 81 (49.39%)             |
| groovy@0  | 8     | 4 (50%)                | 4 (50%)                | 0 (0%)                  | 0 (0%)                  | 0 (0%)                  |
| groovy@1  | 24    | 17 (70.83%)            | 9 (37.5%)              | 14 (58.33%)             | 4 (16.67%)              | 14 (58.33%)             |
| hbase@0   | 106   | 39 (36.79%)            | 39 (36.79%)            | 44 (41.51%)             | 15 (14.15%)             | 44 (41.51%)             |
| hbase@1   | 148   | 65 (43.92%)            | 39 (26.35%)            | 49 (33.11%)             | 32 (21.62%)             | 49 (33.11%)             |
| hive@0    | 78    | 16 (20.51%)            | 5 (6.41%)              | 20 (25.64%)             | 5 (6.41%)               | 20 (25.64%)             |
| hive@1    | 26    | 13 (50%)               | 14 (53.85%)            | 8 (30.77%)              | 15 (57.69%)             | 8 (30.77%)              |
| jruby@0   | 37    | 8 (21.62%)             | 2 (5.41%)              | 21 (56.76%)             | 6 (16.22%)              | 21 (56.76%)             |
| jruby@1   | 44    | 4 (9.09%)              | 1 (2.27%)              | 7 (15.91%)              | 2 (4.55%)               | 7 (15.91%)              |
| jruby@2   | 20    | 8 (40%)                | 10 (50%)               | 6 (30%)                 | 4 (20%)                 | 6 (30%)                 |
| lucene@0  | 128   | 31 (24.22%)            | 14 (10.94%)            | 94 (73.44%)             | 51 (39.84%)             | 93 (72.66%)             |
| lucene@1  | 107   | 62 (57.94%)            | 43 (40.19%)            | 51 (47.66%)             | 34 (31.78%)             | 51 (47.66%)             |
| lucene@2  | 3     | 3 (100%)               | 3 (100%)               | 3 (100%)                | 3 (100%)                | 3 (100%)                |
| wicket@0  | 34    | 22 (64.71%)            | 7 (20.59%)             | 21 (61.76%)             | 14 (41.18%)             | 21 (61.76%)             |
| wicket@1  | 9     | 2 (22.22%)             | 7 (77.78%)             | 4 (44.44%)              | 3 (33.33%)              | 4 (44.44%)              |


### Ex2. Flipping changes' flip rate

| Project    | LIME-HPO    | TimeLIME    | SQAPlanner - conf. | SQAPlanner - cove. | SQAPlanner - lift    |
|------------|-------------|-------------|-------------------------|-----------------------|----------------------|
| activemq@0 | 44          | 33          | 25 (54.35%)             | 16 (34.78%)           | 25 (54.35%)          |
| activemq@1 | 36          | 37          | 12 (50.0%)              | 10 (41.67%)           | 12 (50.0%)           |
| activemq@2 | [...97%]    | [...67%]    | 34 (51.52%)             | 47 (71.21%)           | 34 (51.52%)          |
| activemq@3 | [...]       | [...]       | 23 (29.49%)             | 12 (15.38%)           | 23 (29.49%)          |
| camel@0   | [...97%]    | 32          | 18 (47.37%)             | 15 (39.47%)           | 18 (47.37%)          |
| camel@1   | [...]       | 31          | 6 (54.55%)              | 2 (18.18%)            | 6 (54.55%)           |
| camel@2   | [...]       | 21          | 0 (0.0%)                | 0 (0.0%)              | 0 (0.0%)             |
| derby@0   | [...20%]    | 135         | 239 (60.51%)            | 79 (20.05%)           | 240 (60.76%)         |
| derby@1   | [...]       | [...92%]    | 106 (64.63%)            | 22 (13.41%)           | 103 (62.8%)          |
| groovy@0  | 8           | [...88%]    | 0 (0.0%)                | 2 (25.0%)             | 0 (0.0%)             |
| groovy@1  | 24          | [...]       | 14 (58.33%)             | 3 (12.5%)             | 14 (58.33%)          |
| hbase@0   | 106         | 93          | 52 (49.06%)             | 19 (17.92%)           | 52 (49.06%)          |
| hbase@1   | [...99%]    | 140         | 109 (73.65%)            | 25 (16.89%)           | 109 (73.65%)         |
| hive@0    | [...]       | 17          | 20 (25.64%)             | 9 (11.54%)            | 19 (24.36%)          |
| hive@1    | [...]       | 26          | 10 (41.67%)             | 6 (25.0%)             | 10 (41.67%)          |
| jruby@0   | [...50%]    | 34          | 22 (59.46%)             | 3 (8.11%)             | 22 (59.46%)          |
| jruby@1   | [...]       | [...66%]    | 7 (15.91%)              | 2 (4.55%)             | 7 (15.91%)           |
| jruby@2   | [...]       | [...]       | 8 (40.0%)               | 3 (15.0%)             | 8 (40.0%)            |
| lucene@0  | [...95%]    | 71          | 112 (87.5%)             | 19 (14.84%)           | 110 (85.94%)         |
| lucene@1  | [...]       | 105         | 76 (71.03%)             | 36 (33.64%)           | 76 (71.03%)          |
| lucene@2  | [...]       | 3           | 3 (100.0%)              | 3 (100.0%)            | 3 (100.0%)           |
| wicket@0  | [...]       | 30          | 24 (80.0%)              | 14 (46.67%)           | 24 (80.0%)           |
| wicket@1  | [...]       | 9           | 5 (71.43%)              | 5 (71.43%)            | 5 (71.43%)           |



In [ ]:
# Flip rate of each explainer
import pickle
from data_utils import read_dataset
from hyparams import EXPERIMENTS, MODELS, PLOTS
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

projects = read_dataset()
project_list = list(sorted(projects.keys()))

explainer_list = ['DeFlip', 'LIMEHPO', 'TimeLIME', 'coverage/SQAPlanner', 'confidence/SQAPlanner', 'lift/SQAPlanner']
statistics = {}
for project in project_list:
    _, test = projects[project]
    positives = test.loc[test['target'] == True]
    model_path = Path(MODELS) / project / 'RandomForest.pkl'
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    predictions = model.predict(positives.drop(columns=['target']).values)
    true_positives = positives.loc[predictions == True]
    
    total = len(true_positives)
    for explainer in explainer_list:
        flipped_instances_df = pd.read_csv(Path(EXPERIMENTS) / project / f'{explainer}.csv', index_col=0)
        assert set(flipped_instances_df.index.values).issubset(set(true_positives.index.values))
        flip_rate = len(flipped_instances_df) / total
        statistics[explainer] = statistics.get(explainer, []) + [flip_rate]

df = pd.DataFrame(statistics, index=project_list)
df = df.melt(var_name='Explainer', value_name='Flip Rate')
plt.figure(figsize=(12, 8))
sns.boxplot(x='Explainer', y='Flip Rate', data=df)

# hide the x axis label
plt.xlabel('')

x_labels = ['DeFlip', 'LIME-HPO', 'TimeLIME', 'SQAPlanner[Coverage]', '[Confidence]', '[Lift]']
plt.xticks(range(len(x_labels)), x_labels)

plot_path = Path(PLOTS)
plot_path.mkdir(parents=True, exist_ok=True)
plt.savefig(plot_path / 'flip_rate.png')
plt.show()

#TODO: add statistical test (Scott-Knott ESD) (https://github.com/klainfo/ScottKnottESD)

### Ex2. Fliiping changes' flip rates

> 실험중


### Supplementary: Flipped with Minimal Changes' Feature Count
The red bar indicates the number of features that are flipped by minimal changes when the original feature value was 0.

In [231]:
projects = read_dataset()
project_list = list(sorted(projects.keys()))

explainer_list = ['DeFlip', 'LIMEHPO', 'TimeLIME', 'coverage/SQAPlanner', 'confidence/SQAPlanner', 'lift/SQAPlanner']
statistics = {}
count_statistics = {}
zero_count_statistics = {}
for project in project_list:
    train, test = projects[project]
    mins = train.min()
    maxs = train.max()
    positives = test.loc[test['target'] == True]
    model_path = Path(MODELS) / project / 'RandomForest.pkl'
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    predictions = model.predict(positives.drop(columns=['target']).values)
    true_positives = positives.loc[predictions == True]
    true_positives = true_positives.drop(columns=['target'])
    total = len(true_positives)
    for explainer in explainer_list:
        statistics[explainer] = statistics.get(explainer, {})
        count_statistics[explainer] = count_statistics.get(explainer, {})
        zero_count_statistics[explainer] = zero_count_statistics.get(explainer, {})
        flipped_instances_df = pd.read_csv(Path(EXPERIMENTS) / project / f'{explainer}.csv', index_col=0)
        assert set(flipped_instances_df.index.values).issubset(set(true_positives.index.values))
        if explainer == 'DeFlip':
            flipped_instances_df = flipped_instances_df.drop(columns=['effort'])

        # Let's check out the flipped feature's distributions. 
        # First, we count the number of times each feature is changed when flipped.
        for instance_name in flipped_instances_df.index.values:
            flipped_instance = flipped_instances_df.loc[instance_name]
            original_instance = true_positives.loc[instance_name]
            flipped_features = flipped_instance[flipped_instance != original_instance]
            for feature in flipped_features.index.values:
                if original_instance[feature] == 0:
                    zero_count_statistics[explainer][feature] = zero_count_statistics[explainer].get(feature, 0) + 1
                
                statistics[explainer][feature] = statistics[explainer].get(feature, []) + [flipped_instance[feature] - original_instance[feature]]
                count_statistics[explainer][feature] = count_statistics[explainer].get(feature, 0) + 1

for explainer in explainer_list:
    for feature in count_statistics[explainer].keys():
        if feature not in zero_count_statistics[explainer]:
            zero_count_statistics[explainer][feature] = 0
        zero_count_statistics[explainer][feature] = zero_count_statistics[explainer][feature]

In [ ]:
import numpy as np

for explainer in explainer_list:
    df_count = pd.DataFrame(count_statistics[explainer], index=['count']).T
    df_count = df_count.sort_values(by='count', ascending=False)
    
    fig, ax1 = plt.subplots(figsize=(12, 12))
    
    sns.barplot(x=df_count.index.values, y='count', data=df_count, ax=ax1, label='count')
    plt.xticks(rotation=90)
    ax1.set_ylim([0, 1.1 * df_count['count'].max()])
    ax2 = ax1.twinx()

    df_zero_count = pd.DataFrame(zero_count_statistics[explainer], index=['zero count']).T
    
    df_zero_count = df_zero_count.loc[df_count.index]  # Match the order of features
    df_zero_count = df_zero_count.sort_values(by='zero count', ascending=False)

    sns.barplot(x=df_zero_count.index.values, y='zero count', data=df_zero_count, ax=ax2, label='zero count', color='red')
    for p in ax2.patches:
        p.set_alpha(0.5)
    plt.xticks(rotation=90)
    ax2.set_yticks(ax1.get_yticks())
    ax2.set_ylim(ax1.get_ylim())
    ax2.tick_params(axis='y', which='both', left=False, right=False, labelleft=False, labelright=False)
    ax2.set_ylabel('')

    # legend
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc='upper right')

    plt.title(f'{explainer}: # of times each feature is changed when flipped and # of times original value was zero')
    plt.tight_layout()
    filename = explainer.replace('/', '_')
    plt.savefig(plot_path / f'zero_flips_{filename}.png')
    plt.show()


### Proposed Changes' Feature Count
The red bar indicates the number of features that are proposed by explainers when the original feature value was 0.
The blue bar indicates the number of features that are proposed by explainers when the change percentage was greater than 2000

In [228]:
import json
from hyparams import PLANS

def relative_change_rate(old_value, new_value):
    return (new_value - old_value) / old_value * 100

projects = read_dataset()
project_list = list(sorted(projects.keys()))
baselines = ['LIMEHPO', 'TimeLIME', 'SQAPlanner/confidence', 'SQAPlanner/coverage', 'SQAPlanner/lift']

max_statistics = {}
statistics = {}
count_statistics = {}
for explainer in baselines:
    max_statistics[explainer] = max_statistics.get(explainer, {})
    statistics[explainer] = statistics.get(explainer, {})
    count_statistics[explainer] = count_statistics.get(explainer, {})
    for project in project_list:
        plan_path = Path(PLANS) / project / explainer / 'plans_all.json'
        with open(plan_path, 'r') as f:
            plans = json.load(f)

        train, test = projects[project]
        test_instances = test.drop(columns=['target'])
        mins = train.min()
        maxs = train.max()

        for instance_name in plans.keys():
            plan = plans[instance_name]
            origin = test_instances.loc[int(instance_name)]
            for feature in plan.keys():
                if origin[feature] == 0:
                    statistics[explainer][feature] = statistics[explainer].get(feature, 0) + 1
                elif relative_change_rate(origin[feature], max(plan[feature])) > 2000:
                    max_statistics[explainer][feature] = max_statistics[explainer].get(feature, 0) + 1
                count_statistics[explainer][feature] = count_statistics[explainer].get(feature, 0) + 1

for explainer in baselines:
    for feature in count_statistics[explainer].keys():
        if feature not in statistics[explainer]:
            statistics[explainer][feature] = 0
        if feature not in max_statistics[explainer]:
            max_statistics[explainer][feature] = 0
        statistics[explainer][feature] = statistics[explainer][feature]

In [ ]:
# Zero and Max count

for explainer in baselines:
    df_count = pd.DataFrame(count_statistics[explainer], index=['count']).T
    df_count = df_count.sort_values(by='count', ascending=False)
    
    fig, ax1 = plt.subplots(figsize=(12, 12))
    
    sns.barplot(x=df_count.index.values, y='count', data=df_count, ax=ax1, label='count')
    plt.xticks(rotation=90)
    ax1.set_ylim([0, 1.1 * df_count['count'].max()])
    ax2 = ax1.twinx()
    ax3 = ax1.twinx()

    df_zero_count = pd.DataFrame(statistics[explainer], index=['zero_count']).T
    
    df_zero_count = df_zero_count.loc[df_count.index]  # Match the order of features
    df_zero_count = df_zero_count.sort_values(by='zero_count', ascending=False)

    sns.barplot(x=df_zero_count.index.values, y='zero_count', data=df_zero_count, ax=ax2, label='zero_count', color='red', width=0.4)
    for p in ax2.patches:
        p.set_alpha(0.5)
        x = p.get_x() - 0.2
        p.set_x(x)
    plt.xticks(rotation=90)
    ax2.set_yticks(ax1.get_yticks())
    ax2.set_ylim(ax1.get_ylim())
    ax2.tick_params(axis='y', which='both', left=False, right=False, labelleft=False, labelright=False)
    ax2.set_ylabel('')

    df_max_count = pd.DataFrame(max_statistics[explainer], index=['max_count']).T
    
    df_max_count = df_max_count.loc[df_count.index]  # Match the order of features
    df_max_count = df_max_count.sort_values(by='max_count', ascending=False)

    sns.barplot(x=df_max_count.index.values, y='max_count', data=df_max_count, ax=ax3, label='max_count', color='blue', width=0.4)
    for p in ax3.patches:
        p.set_alpha(0.5)
        x = p.get_x() + 0.2
        p.set_x(x)
    plt.xticks(rotation=90)
    ax3.set_yticks(ax1.get_yticks())
    ax3.set_ylim(ax1.get_ylim())
    ax3.tick_params(axis='y', which='both', left=False, right=False, labelleft=False, labelright=False)
    ax3.set_ylabel('')

    # legend
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    h3, l3 = ax3.get_legend_handles_labels()
    ax1.legend(h1+h2+h3, l1+l2+l3, loc='upper right')

    plt.title(f'{explainer}: # of times each feature is changed when flipped and # of times original value was zero')
    plt.tight_layout()
    filename = explainer.replace('/', '_')
    plt.savefig(plot_path / f'all_plans_{filename}.png')
    plt.show()


In [ ]:
dfs = []

for explainer in baselines:
    for feature, values in statistics[explainer].items():
        temp_df = pd.DataFrame({
            'Feature': [feature] * len(values),
            'max_change_rate': values
        })
        dfs.append(temp_df)

    df = pd.concat(dfs, ignore_index=True)
    
    plt.figure(figsize=(12, 12))
    ax = sns.boxplot(x='Feature', y='max_change_rate', data=df)

    # plot median values to circle
    medians = df.groupby(['Feature'])['max_change_rate'].median().values

    for idx, patch in enumerate(ax.artists):
        y = medians[idx]
        x = patch.get_x() + patch.get_width() / 2
        ax.plot(x, y, 'o', fillstyle='none',  markersize=10)
    
    # hide the x axis label
    plt.xlabel('')
    plt.xticks(rotation=90)
    # plt.ylim([0, 1000])
    plt.title(f'{explainer}: max_change_rate of each feature')
    plt.tight_layout()
    filename = explainer.replace('/', '_')
    plt.savefig(plot_path / f'change_rate_{filename}.png')
    plt.show()


In [ ]:
# max chage ratio of each feature
for explainer in baselines:
    
